# 0. Setup Paths

In [1]:
import os

In [2]:
CUSTOM_MODEL_NAME = 'Final_Module-3' 
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [3]:
paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join('Tensorflow','protoc')
 }

In [4]:
files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [5]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

# 2. Create Label Map

In [6]:
labels = [{'name':'thumbsup', 'id':1}, {'name':'thumbsdown', 'id':2},{'name':'Ali', 'id':3},{'name':'LightUp', 'id':4},{'name':'LightDown', 'id':5}]
with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# 8. Load Train Model From Checkpoint

In [7]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

In [8]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], 'ckpt-4')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

In [9]:

import cv2 
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [10]:
category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])

In [11]:
import serial.tools.list_ports

def get_ports():

    ports = serial.tools.list_ports.comports()
    
    return ports

def findArduino(portsFound):
    
    commPort = 'None'
    numConnection = len(portsFound)
    
    for i in range(0,numConnection):
        port = foundPorts[i]
        strPort = str(port)
        
        if 'USB' in strPort: 
            splitPort = strPort.split(' ')
            commPort = (splitPort[0])

    return commPort
            
                    
foundPorts = get_ports()        
connectPort = findArduino(foundPorts)

if connectPort != 'None':
    ser = serial.Serial(connectPort,baudrate = 9600, timeout=1)
    print('Connected to ' + connectPort)

else:
    print('Connection Issue!')

print('DONE')

Connected to COM5
DONE


In [ ]:
import socket
import threading
import sys
import time

class Server :
    sock = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
    connections = []
    def __init__(self):
        self.sock.bind(('0.0.0.0',10000))
        self.sock.listen(1)
    def handler(self,c,a):
        while True:
            data = c.recv(1024)
            for connection in self.connections:
                connection.send(data)
            if not data:
                self.connections.remove(c)
                c.close()
                break
    def run(self):
        while True:
            c, a =self.sock.accept()
            cThread = threading.Thread(target=self.handler,args=(c, a))
            cThread.daemon = True
            cThread.start()
            self.connections.append(c)
            print(self.connections)
class Client():
    
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    def getdMsg(self):
          while True:
                data = self.sock.recv(1024)
                if not data:
                    break
        
       
    def __init__(self,address):
        address ="192.168.1.105"
        self.sock.connect((address,8888))
        
        iThread = threading.Thread(target=self.getMsg)
        iThread.daemon = True
        iThread.start()


        cap = cv2.VideoCapture(0)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        y = 0

        while cap.isOpened():
            ret, frame = cap.read()
            frame = cv2.flip(frame, 1)
            image_np = np.array(frame)

            input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
            detections = detect_fn(input_tensor)

            num_detections = int(detections.pop('num_detections'))
            detections = {key: value[0, :num_detections].numpy()
                          for key, value in detections.items()}
            detections['num_detections'] = num_detections
            # detection_classes should be ints.
            detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

            label_id_offset = 1
            image_np_with_detections = image_np.copy()

            label, _ = viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes'] + label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.8,
                agnostic_mode=False)
            
            Mem = label.split(":")
            
            
            if Mem[0] == 'Ali':
                y = 1
            elif Mem[0] == '':
                y = 0
            else:
                pass
            if y == 1:
                if Mem[0] == 'thumbsup':
                    ser.write(b'1')
                if Mem[0] == 'thumbsdown':
                    ser.write(b'2')
                if Mem[0] == 'LightUp':
                    ser.write(b'3')
                if Mem[0] == 'LightDown':
                    ser.write(b'4')
            else:
                cv2.putText(image_np_with_detections, 'Yetkisiz Giris', (250, 450), cv2.FONT_HERSHEY_SIMPLEX, 1,
                            (0, 0, 0), 2)
            cv2.imshow('object detection', cv2.resize(image_np_with_detections, (800, 600)))

            if cv2.waitKey(10) & 0xFF == ord('q'):
                cap.release()
                cv2.destroyAllWindows()
                break

if(len(sys.argv)>1):
    client =Client(sys.argv[1])
else:
    server = Server()
    server.run()

In [12]:
import socket
import threading
import sys

class Server :
    sock = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
    connections = []
    def __init__(self):
        self.sock.bind(('192.168.1.105',8888))
        self.sock.listen(1)
    def handler(self,c,a):
        while True:
            data = c.recv(1024)
            for connection in self.connections:
                connection.send(data)
            if not data:
                self.connections.remove(c)
                c.close()
                break
    def run(self):
        while True:
            c, a =self.sock.accept()
            cThread = threading.Thread(target=self.handler,args=(c, a))
            cThread.daemon = True
            cThread.start()
            self.connections.append(c)
            print(self.connections)
class Client():
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    def sendMsg(self):
        cap = cv2.VideoCapture(0)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        y = 0

        while cap.isOpened():
            ret, frame = cap.read()
            frame = cv2.flip(frame, 1)
            image_np = np.array(frame)

            input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
            detections = detect_fn(input_tensor)

            num_detections = int(detections.pop('num_detections'))
            detections = {key: value[0, :num_detections].numpy()
                          for key, value in detections.items()}
            detections['num_detections'] = num_detections
            # detection_classes should be ints.
            detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

            label_id_offset = 1
            image_np_with_detections = image_np.copy()

            label, _ = viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes'] + label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.8,
                agnostic_mode=False)
            
            Mem = label.split(":")
            
            
            if Mem[0] == 'Ali':
                y = 1
            elif Mem[0] == '':
                y = 0
            else:
                pass
            if y == 1:
                if Mem[0] == 'thumbsup':
                    ser.write(b'1')
                if Mem[0] == 'thumbsdown':
                    ser.write(b'2')
                if Mem[0] == 'LightUp':
                    ser.write(b'3')
                if Mem[0] == 'LightDown':
                    ser.write(b'4')
            else:
                cv2.putText(image_np_with_detections, 'Yetkisiz Giris', (250, 450), cv2.FONT_HERSHEY_SIMPLEX, 1,
                            (0, 0, 0), 2)
            cv2.imshow('object detection', cv2.resize(image_np_with_detections, (800, 600)))

            if cv2.waitKey(10) & 0xFF == ord('q'):
                cap.release()
                cv2.destroyAllWindows()
                break

    def __init__(self,address):
        address ="192.168.1.105"
        self.sock.connect((address,8888))

        iThread = threading.Thread(target=self.sendMsg)
        iThread.daemon = True
        iThread.start()

        while True:
            data = self.sock.recv(1024)
            if data == b'Channelup':
                ser.write(b'1')
            if data == b'ChannelDown':
                ser.write(b'2')
            if data == b'LightUp':
                ser.write(b'3')
            if data == b'LightDown':
                ser.write(b'4')    
            if not data :
                break
            print(data)
if(len(sys.argv)>1):
    client =Client(sys.argv[1])
else:
    server = Server()
    server.run()



b'LightUp'
b'LightUp'
b'LightUp'
b'LightUp'
b'LightUp'
b'LightUp'
b'LightDown'
b'LightDown'
b'LightDown'
b'LightDown'
b'LightDown'
b'LightDown'
b'Channelup'
b'Channelup'
b'Channelup'
b'Channelup'
b'ChannelDown'
b'ChannelDown'
b'ChannelDown'


ConnectionResetError: [WinError 10054] Varolan bir bağlantı uzaktaki bir ana bilgisayar tarafından zorla kapatıldı

In [ ]:
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
y = 0

while cap.isOpened(): 
    ret, frame = cap.read()
    frame= cv2.flip(frame,1)
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    label,_=viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.8,
                agnostic_mode=False)
    
    x = label.split(":")
    print(x)
    if x[0]=='Ali':
        y=1
    elif x[0]=='':
        y=0
    else:
        pass
    if y==1:
        if x[0]=='thumbsup':
            ser.write(b'1')
        if x[0]=='thumbsdown':
            ser.write(b'2')
        if x[0]=='LightUp':
            ser.write(b'3')
        if x[0]=='LightDown':
            ser.write(b'4')
    else:
        cv2.putText(image_np_with_detections,'Yetkisiz Giris',(250,450),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),2)   
    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break
